In [39]:
import pandas as pd
import requests
import json
import threading
from time import sleep


In [49]:
headers={
    'user-agent':'Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1',
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
}
params={
    'limit': '10',
    'include': 'advertisement',
    'is_mweb': '1',
    'aggregations': '2',
    'q': 'điện thoại samsung',
    'page': '1'
}
product_list=[]

In [41]:

def Get_List_ID(page):
    params['pages']=page
    response=requests.get('https://tiki.vn/api/v2/products',headers=headers,params=params)
    if response.status_code == 200:
        print(f'-------Crawl page {page} successfully!!!--------')
    for record in response.json().get('data'):
        product_list.append(record.get('id'))

In [52]:
def Get_List_ID(page):
    params['pages']=page
    response=requests.get('https://tiki.vn/api/v2/products',headers=headers,params=params)
    if response.status_code == 200:
        print(f'-------Crawl page {page} successfully!!!--------')
    for record in response.json().get('data'):
        product_list.append(record.get('id'))
        
def Get_List_productID():
    threads=[]
    for i in range(1,10):
        sleep(0.2)
        t=threading.Thread(target=Get_List_ID,args=(i,))
        threads.append(t)
        
    for thread in threads:
        thread.start()
        
    return product_list


In [24]:
import pandas as pd
import requests
import json
import threading
from time import sleep
from sqlalchemy import create_engine

In [35]:

headers={
'user-agent':'Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1',
'accept': 'application/json, text/plain, */*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
}

params={
    'limit': '15',
    'include': 'advertisement',
    'is_mweb': '1',
    'aggregations': '2',
    'q': 'điện thoại samsung',
    'page':1
}

#MySQL connection information
username="root"
password=""
host="localhost"
database="tiki"
engine=create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
product_list=[]

In [36]:

def Get_List_ID(page):
    params['pages']=page
    response=requests.get('https://tiki.vn/api/v2/products',headers=headers,params=params)
    if response.status_code == 200:
        print(f'-------Crawl page {page} successfully!!!--------')
    for record in response.json().get('data'):
        product_list.append(record.get('id'))
        

In [37]:
def Get_List_productID():
    threads=[]
    for i in range(1,2):
        sleep(0.1)
        t=threading.Thread(target=Get_List_ID,args=(i,))
        threads.append(t)
        
    for thread in threads:
        thread.start()
    

In [38]:
Get_List_productID()

-------Crawl page 1 successfully!!!--------


In [39]:
product_list

[127372598,
 197670818,
 102077313,
 177141832,
 162593187,
 162593114,
 194034217,
 172001933,
 173764269,
 170581659,
 172826900,
 175967811,
 162593136,
 158741127,
 177746939]

In [33]:
def Get_Product_Information():
    masterProduct=[]
    productDetail=[]
    Marketing=[]
    for id in product_list:
        response=requests.get(f'https://tiki.vn/api/v2/products/{id}',headers=headers,params=params)
        text=response.content
        data=json.loads(text)

        master_=[]
        master_.append(data['id'])
        master_.append(data['name'])
        master_.append(data['categories']['name'])
        master_.append(data['brand']['name'])
        master_.append(data['current_seller']['id'])
        master_.append(data['current_seller']['name'])
        masterProduct.append(master_)
        
        for i in range(0,len(data['configurable_products'])):
            detail_=[]
            detail_.append(data['id'])
            detail_.append(int(data['configurable_products'][i]['sku']))
            detail_.append(data['configurable_products'][i]['option1'])
            detail_.append(data['configurable_products'][i]['price'])
            productDetail.append(detail_)
            
        marketing_=[]
        marketing_.append(data['id'])
        marketing_.append(data['review_count'])
        marketing_.append(data['rating_average'])
        marketing_.append(data['all_time_quantity_sold'])
        Marketing.append(marketing_)

    if masterProduct is not None:
        if productDetail is not None and Marketing is not None:
            print('------Data is available-------')
    df_MasterProduct=pd.DataFrame(data=masterProduct,columns=['ID','Name','Category','Brand','Shop ID','Shop name'])
    df_ProductDetail=pd.DataFrame(data=productDetail,columns=['Product ID','SKU','Option','Price'])
    df_Marketing=pd.DataFrame(data=Marketing,columns=['Product ID','Count review','Average score','Sold items'])
    
    print('-------Crawl data successfully-------')
    # Dataframe to Excel 
    # df_MasterProduct.to_excel('Master Product.xlsx')
    # df_ProductDetail.to_excel('Product Detail.xlsx')
    # df_Marketing.to_excel('Marketing.xlsx')
    
    #DataFrame to MySQL
    df_MasterProduct.to_sql('master product',con=engine,if_exists='append',index=False)
    df_ProductDetail.to_sql('product detail',con=engine,if_exists='append',index=False)
    df_Marketing.to_sql('marketing',con=engine,if_exists='append',index=False)



In [34]:
Get_Product_Information(

------Data is available-------
-------Crawl data successfully-------


In [ ]:
mas

In [6]:




Get_List_productID()
print(product_list)
# Get_Product_Information()

[]


-------Crawl page 1 successfully!!!--------
